In [2]:
#Prabhat Thakur Date 06/07/2019
#2019SP_MSDS_458-DL_SEC56 Week10- Assignment4
#Create deep neural networks CNN for Gender classcification.
#Data : Downloaded from Google 
#Code Addopted from F. Challot (2018), Deep Learning with Python (Manning) 
#              and https://github.com/arunponnusamy/gender-detection-keras

In [3]:
import keras
from keras.models import Sequential,load_model
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D,MaxPooling2D
from keras.layers.core import Dense,Activation,Flatten,Dropout
from keras import backend as K

from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from keras.utils import to_categorical
from keras.utils import plot_model


In [4]:
import matplotlib
matplotlib.use("Agg")

from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob
import cvlib as cv

In [5]:
def welcome ():
    print (' ' )
    print ('******************************************************************************')
    print (' ' )
    print ('This program is for Gender classcification using CNN network')
    print ('deep learning neural network structures. ')
    print ('Dataset is gathered from Google Images .')
    print ('Classifying Gender: a binary classification example')
    print (' ' )
    print ('******************************************************************************')
    print (' ' )
    return()

In [6]:
# to make this notebook's output stable across runs
np.random.seed(42)

epochs = 50
lr= 0.001
batch_size = 64
img_dims = (96,96,3)

data = []
labels = []

# load image files from the dataset
image_files = [f for f in glob.glob('C:/Users/prabhatkumar/Documents/MSDS458Assignment4/gender_dataset_face' + "/**/*", recursive=True) if not os.path.isdir(f)] 
random.shuffle(image_files)

In [7]:
# create groud-truth label from the image path
for img in image_files:

    image = cv2.imread(img)
    
    image = cv2.resize(image, (img_dims[0],img_dims[1]))
    image = img_to_array(image)
    data.append(image)

    label = img.split(os.path.sep)[-2]
    if label == "woman":
        label = 1
    else:
        label = 0
        
    labels.append([label])

# pre-processing
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)

# split dataset for training and validation
(trainX, testX, trainY, testY) = train_test_split(data, labels, test_size=0.2,
                                                  random_state=42)
trainY = to_categorical(trainY, num_classes=2)
testY = to_categorical(testY, num_classes=2)


In [8]:
# augmenting datset 
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
                         height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
                         horizontal_flip=True, fill_mode="nearest")

In [9]:
def plot_loss (history):
    history_dict = history.history
    loss = history.history['loss']
    val_loss = history.history['val_loss']

    epochs = range(1, len(loss) + 1)

    # "bo" is for "blue dot"
    plt.plot(epochs, loss, 'bo', label='Training loss')
    # b is for "solid blue line"
    plt.plot(epochs, val_loss, 'b', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()

In [10]:
def plot_accuracy (history):
    history_dict = history.history
    
    acc = history_dict['acc']
    val_acc = history_dict['val_acc']
    
    plt.clf()
    
    epochs = range(1, len(acc) + 1)
    
    plt.plot(epochs, acc, 'bo', label='Training acc')
    plt.plot(epochs, val_acc, 'b', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.show()

In [11]:
class SmallerVGGNet:
    @staticmethod
    def build(width, height, depth, classes):
        model = Sequential()
        inputShape = (height, width, depth)
        chanDim = -1

        if K.image_data_format() == "channels_first":
            inputShape = (depth, height, width)
            chanDim = 1

        model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(3,3)))
        model.add(Dropout(0.25))

        model.add(Conv2D(64, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(64, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))

        model.add(Conv2D(128, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(Conv2D(128, (3,3), padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(axis=chanDim))
        model.add(MaxPooling2D(pool_size=(2,2)))
        model.add(Dropout(0.25))

        model.add(Flatten())
        model.add(Dense(1024))
        model.add(Activation("relu"))
        model.add(BatchNormalization())
        model.add(Dropout(0.5))

        model.add(Dense(classes))
        model.add(Activation("sigmoid"))

        return model

In [12]:
# build model
GenderModel = SmallerVGGNet()
model = GenderModel.build(width=img_dims[0], height=img_dims[1], depth=img_dims[2],
                          classes=2) 
    
# compile the model
opt = Adam(lr=lr, decay=lr/epochs)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])


In [13]:
# train the model
H = model.fit_generator(aug.flow(trainX, trainY, batch_size=batch_size),
                        validation_data=(testX,testY),
                        steps_per_epoch=len(trainX) // batch_size,
                        epochs=epochs, verbose=1)

model.save('gender_prediction1.model')

Epoch 1/50
28/28 [==============================] - 251s 9s/step - loss: 0.7901 - acc: 0.7116 - val_loss: 0.6170 - val_acc: 0.7727
Epoch 2/50
28/28 [==============================] - 282s 10s/step - loss: 0.4767 - acc: 0.8003 - val_loss: 0.5124 - val_acc: 0.8084
Epoch 3/50
28/28 [==============================] - 295s 11s/step - loss: 0.4171 - acc: 0.8306 - val_loss: 0.4303 - val_acc: 0.8323
Epoch 4/50
28/28 [==============================] - 291s 10s/step - loss: 0.3687 - acc: 0.8457 - val_loss: 0.3714 - val_acc: 0.8604
Epoch 5/50
28/28 [==============================] - 292s 10s/step - loss: 0.3056 - acc: 0.8758 - val_loss: 0.3352 - val_acc: 0.8766
Epoch 6/50
28/28 [==============================] - 290s 10s/step - loss: 0.2823 - acc: 0.8895 - val_loss: 0.7001 - val_acc: 0.7522
Epoch 7/50
28/28 [==============================] - 291s 10s/step - loss: 0.2949 - acc: 0.8784 - val_loss: 0.3991 - val_acc: 0.8496
Epoch 8/50
28/28 [==============================] - 291s 10s/step - loss: 0.2

In [14]:
welcome ()
model.summary()

 
******************************************************************************
 
This program is for Gender classcification using CNN network
deep learning neural network structures. 
Dataset is gathered from Google Images .
Classifying Gender: a binary classification example
 
******************************************************************************
 
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 96, 96, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 96, 96, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 96, 32)        128       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 32)        0         
____________________________________________

In [21]:
# plot training/validation loss/accuracy
plt.style.use("ggplot")
plt.figure()
N = epochs
plt.plot(np.arange(0,N), H.history["loss"], label="train_loss")
plt.plot(np.arange(0,N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,N), H.history["acc"], label="train_acc")
plt.plot(np.arange(0,N), H.history["val_acc"], label="val_acc")

plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")

plt.show()
# save plot to disk
plt.savefig('train_val.jpg')


In [23]:
import pydot

from keras.utils.vis_utils import model_to_dot
keras.utils.vis_utils.pydot = pydot

plot_model(model, to_file='gender_prediction1.png', show_shapes=True, show_layer_names=True)


In [25]:
# Lets test our model
# read input image
image = cv2.imread('Prabhat.JPG')

if image is None:
    print("Could not read input image")
    exit()

# load pre-trained model
saved_model = load_model('gender_prediction.model')

# detect faces in the image
face, confidence = cv.detect_face(image)

classes = ['man','woman']

# loop through detected faces
for idx, f in enumerate(face):

     # get corner points of face rectangle       
    (startX, startY) = f[0], f[1]
    (endX, endY) = f[2], f[3]

    # draw rectangle over face
    cv2.rectangle(image, (startX,startY), (endX,endY), (0,255,0), 2)

    # crop the detected face region
    face_crop = np.copy(image[startY:endY,startX:endX])

    # preprocessing for gender detection model
    face_crop = cv2.resize(face_crop, (96,96))
    face_crop = face_crop.astype("float") / 255.0
    face_crop = img_to_array(face_crop)
    face_crop = np.expand_dims(face_crop, axis=0)

    # apply gender detection on face
    conf = saved_model.predict(face_crop)[0]
    print(conf)
    print(classes)

    # get label with max accuracy
    idx = np.argmax(conf)
    label = classes[idx]

    label = "{}: {:.2f}%".format(label, conf[idx] * 100)

    Y = startY - 10 if startY - 10 > 10 else startY + 10

    # write label and confidence above face rectangle
    cv2.putText(image, label, (startX, Y),  cv2.FONT_HERSHEY_SIMPLEX,
                0.7, (0, 255, 0), 2)

# display output
cv2.imshow("gender detection", image)
cv2.imwrite( "Prabhat_result.jpg", image );

[9.999957e-01 4.959139e-06]
['man', 'woman']
